In [16]:
import streamlit as st
from PIL import Image
import requests
import io
import pandas as pd
from filter_df import filter_dataframe

AttributeError: partially initialized module 'streamlit' has no attribute 'set_page_config' (most likely due to a circular import)

In [18]:
# API URL for vegetable recognition
api_url = "https://veggideas-5s5h4zi4hq-ew.a.run.app"

img_file_buffer = st.file_uploader('Upload an image')

if img_file_buffer is not None:
    # Display the image uploaded by the user
    image = Image.open(img_file_buffer)

    # Make predictions when the user clicks the button
    if st.button('Make Predictions'):
        with st.spinner("Wait for it..."):
            # Send the image to the API endpoint
            # convert the PIL image to byte array
            image_bytes = io.BytesIO()
            image.save(image_bytes, format='JPEG')
            image_bytes = image_bytes.getvalue()
            api_url = api_url + "/predict"
            response = requests.post(api_url, files={'img': image_bytes})
            data = response.json()
            df = pd.DataFrame(data)

NameError: name 'st' is not defined

In [ ]:
def filter_dataframe(df: dict) -> dict:
    """
    Adds a UI on top of a dataframe to let viewers filter columns

    Args:
        df (dict): Original dataframe as a dictionary of recipe details

    Returns:
        dict: Filtered dataframe as a dictionary
    """
    modify = st.checkbox("Add filters")

    if not modify:
        return df

    df = df.copy()

    modification_container = st.container()

    with modification_container:
        to_filter_columns = st.multiselect("Filter dataframe on", list(df.keys()))
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            # Treat columns with < 10 unique values as categorical
            if isinstance(df[column], list) or len(set(df[column])) < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    list(set(df[column])),
                    default=list(set(df[column])),
                )
                df = {k: v for k, v in df.items() if k == column or v in user_cat_input}
            elif all(isinstance(val, (int, float)) for val in df[column]):
                _min = float(min(df[column]))
                _max = float(max(df[column]))
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    min_value=_min,
                    max_value=_max,
                    value=(_min, _max),
                    step=step,
                )
                df = {k: v for k, v in df.items() if k == column or (isinstance(v, (int, float)) and v >= user_num_input[0] and v <= user_num_input[1])}
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = {k: v for k, v in df.items() if k == column or (isinstance(v, str) and user_text_input in v)}

    return df

In [ ]:
import streamlit as st
from PIL import Image
import requests
import io
import pandas as pd
from filter_df import filter_dataframe

# Set page tab display
st.set_page_config(
    page_title="Simple Image Uploader",
    page_icon='🖼️',
    layout="wide",
    initial_sidebar_state="expanded",
)

# API URL for vegetable recognition
api_url = "https://veggideas-5s5h4zi4hq-ew.a.run.app"

# App title and description
st.header('Simple Image Uploader 📸')
st.markdown('''
            > This is a Le Wagon boilerplate for any data science projects that involve exchanging images between a Python API and a simple web frontend.

            > **What's here:**

            > * [Streamlit](https://docs.streamlit.io/) on the frontend
            > * [FastAPI](https://fastapi.tiangolo.com/) on the backend
            > * [PIL/pillow](https://pillow.readthedocs.io/en/stable/) and [opencv-python](https://github.com/opencv/opencv-python) for working with images
            > * Backend and frontend can be deployed with Docker
            ''')

st.markdown("---")

@st.cache(allow_output_mutation=True)
def get_session_state():
    return {
        'filter_columns': [],
    }

### Create a native Streamlit file upload input
st.markdown("### Let's do vegetable recognition 👇")
img_file_buffer = st.file_uploader('Upload an image')

if img_file_buffer is not None:
    # Display the image uploaded by the user
    image = Image.open(img_file_buffer)
    st.image(image, caption="Here's the image you uploaded ☝️")
    #st.write("Any other preferences?")
    #vegetarian = st.checkbox('vegetarian')
    #vegan = st.checkbox('vegan')
    #pescatarian = st.checkbox('pescatarian')
    #surprise = st.checkbox('surprise me')
    # Make predictions when the user clicks the button
    modify = st.checkbox("Add filters")

    if st.button('Make Predictions'):
        with st.spinner("Wait for it..."):
            # Send the image to the API endpoint
            image_bytes = img_file_buffer.getvalue()

            st.write("Sending image to the API...")

            # Use 'rb' if you get an error about 'bytes-like object is required, not str'
            api_url = api_url + "/predict"
            response = requests.post(api_url, files={'img': image_bytes})
            #response = st.cache_data(response)
            st.write(response.status_code)
            st.write(response.raise_for_status)
            if response.status_code == 200:
                # Parse the predictions from the JSON response
                data = response.json()
                # Create a DataFrame from the predictions data
                df = pd.DataFrame(data)
                st.session_state.df = df

                # Create a DataFrame from the predictions data
                if modify:
                    session_state = get_session_state()

                    to_filter_columns = st.multiselect(
                        "Filter dataframe on",
                        list(df.columns),
                        default=session_state['filter_columns']
                    )

                    session_state['filter_columns'] = to_filter_columns

                    if len(to_filter_columns) > 0:
                        df = filter_dataframe(df, to_filter_columns)
                st.dataframe(df)

                """if vegetarian:
                    st.dataframe(df[df['Diet Type'].apply(lambda x: 'Vegetarian' in x)])
                if vegan:
                    st.dataframe(df[df['Diet Type'].apply(lambda x: 'Vegan' in x)])
                if pescatarian:
                    st.dataframe(df[df['Diet Type'].apply(lambda x: 'Pescatarian' in x)])
                if vegetarian and vegan:
                    st.dataframe(df[df['Diet Type'].apply(lambda x: 'Vegan' and 'Vegetarian' in x)])
                if surprise:
                    st.dataframe(df)"""

            else:
                st.error("Error making predictions. Please try again.")


In [ ]:
import pandas as pd
from pandas.api.types import is_categorical_dtype, is_numeric_dtype
import streamlit as st

def filter_dataframe(df: dict, to_filter_columns) -> dict:
    """
    Adds a UI on top of a dataframe to let viewers filter columns

    Args:
        df (dict): Original dataframe as a dictionary of recipe details

    Returns:
        dict: Filtered dataframe as a dictionary
    """
    df = df.copy()

    modification_container = st.container()

    with modification_container:
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            # Treat columns with < 10 unique values as categorical
            if isinstance(df[column], list) or len(set(df[column])) < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    list(set(df[column])),
                    default=list(set(df[column])),
                )
                df = {k: v for k, v in df.items() if k == column or v in user_cat_input}
            elif all(isinstance(val, (int, float)) for val in df[column]):
                _min = float(min(df[column]))
                _max = float(max(df[column]))
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    min_value=_min,
                    max_value=_max,
                    value=(_min, _max),
                    step=step,
                )
                df = {k: v for k, v in df.items() if k == column or (isinstance(v, (int, float)) and v >= user_num_input[0] and v <= user_num_input[1])}
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = {k: v for k, v in df.items() if k == column or (isinstance(v, str) and user_text_input in v)}

        return df
